# 00 Single-Factor Regression (Micro)

Single-factor log-log regression on county data; interpret coefficients.


## Table of Contents
- [Load census data](#load-census-data)
- [Build log variables](#build-log-variables)
- [Fit OLS + HC3](#fit-ols-hc3)
- [Interpretation](#interpretation)
- [Checkpoint (Self-Check)](#checkpoint-self-check)
- [Solutions (Reference)](#solutions-reference)


## Why This Notebook Matters
Regression is the bridge between statistics and ML. You will learn:
- single-factor vs multi-factor interpretation,
- robust standard errors,
- coefficient stability and multicollinearity.


## What You Will Produce
- (no file output; learning/analysis notebook)

## Success Criteria
- You can explain what you built and why each step exists.
- You can run your work end-to-end without undefined variables.

## Common Pitfalls
- Running cells top-to-bottom without reading the instructions.
- Leaving `...` placeholders in code cells.
- Treating coefficients as causal without a causal design.
- Ignoring multicollinearity (unstable coefficients).

## Matching Guide
- `docs/guides/02_regression/00_single_factor_regression_micro.md`



## How To Use This Notebook
- This notebook is hands-on. Most code cells are incomplete on purpose.
- Complete each TODO, then run the cell.
- Use the matching guide (`docs/guides/02_regression/00_single_factor_regression_micro.md`) for deep explanations and alternative examples.
- Write short interpretation notes as you go (what changed, why it matters).



<a id="environment-bootstrap"></a>
## Environment Bootstrap
Run this cell first. It makes the repo importable and defines common directories.



In [ ]:
from __future__ import annotations

from pathlib import Path
import sys


def find_repo_root(start: Path) -> Path:
    p = start
    for _ in range(8):
        if (p / 'src').exists() and (p / 'docs').exists():
            return p
        p = p.parent
    raise RuntimeError('Could not find repo root. Start Jupyter from the repo root.')


PROJECT_ROOT = find_repo_root(Path.cwd())
if str(PROJECT_ROOT) not in sys.path:
    sys.path.append(str(PROJECT_ROOT))

DATA_DIR = PROJECT_ROOT / 'data'
RAW_DIR = DATA_DIR / 'raw'
PROCESSED_DIR = DATA_DIR / 'processed'
SAMPLE_DIR = DATA_DIR / 'sample'

PROJECT_ROOT



## Goal
Fit a single-factor log-log regression on county data and interpret the coefficient like an elasticity.

Example question:
- "Across counties, how does rent scale with income?"

This is **not** a causal claim by default. It's a structured description of a relationship.



## Primer: statsmodels vs scikit-learn (Inference vs Prediction)

### Two different goals
- **Prediction (ML):** optimize out-of-sample accuracy.
- **Inference (econometrics):** interpret coefficients + quantify uncertainty.

scikit-learn is mostly prediction-focused.
statsmodels is built for inference (standard errors, p-values, confidence intervals).

### Minimal statsmodels OLS pattern
```python
import statsmodels.api as sm

# X: DataFrame of features
# y: Series target

# Add intercept column
# Xc = sm.add_constant(X, has_constant='add')

# Fit OLS
# res = sm.OLS(y, Xc).fit()
# print(res.summary())
```

### What you're looking at in `res.summary()`
Typical regression output includes:
- **coef**: estimated coefficients $\hat\beta$
- **std err**: estimated standard errors $\widehat{SE}(\hat\beta)$
- **t**: t-statistic $\hat\beta / \widehat{SE}(\hat\beta)$
- **P>|t|**: p-value for $H_0: \beta=0$ (under assumptions)
- **[0.025, 0.975]**: 95% confidence interval for $\beta$

You can access these programmatically:

```python
# res.params        # coefficients (pandas Series)
# res.bse           # standard errors
# res.pvalues       # p-values
# res.conf_int()    # confidence intervals
```

### Robust standard errors
Robust SE change uncertainty estimates, not coefficients.

```python
# HC3 (cross-section)
# res_hc3 = res.get_robustcov_results(cov_type='HC3')

# HAC/Newey-West (time series)
# res_hac = res.get_robustcov_results(cov_type='HAC', cov_kwds={'maxlags': 4})
```

Why this matters:
- Cross-sectional data often has **heteroskedasticity** (error variance changes across observations).
- Time series often has **autocorrelation** (errors correlated over time).

### Practical rule
- Use sklearn for predictive pipelines and cross-validation.
- Use statsmodels when you need inference and careful interpretation.


## Primer: Hypothesis Testing (p-values, t-tests, and Confidence Intervals)

You will see p-values, t-statistics, and confidence intervals in regression output (especially `statsmodels`).
This primer gives you the minimum you need to avoid the most common misunderstandings.

### Definitions (in plain language)
- **Hypothesis**: a claim about an unknown population quantity (a parameter).
- **Null hypothesis** $H_0$: the "default" claim (often “no effect”).
- **Alternative hypothesis** $H_1$: the claim you consider if the data looks inconsistent with $H_0$.
- **Test statistic**: a number computed from data that measures how surprising the data is under $H_0$.
- **p-value**: the probability (under the assumptions of the null model) of seeing a test statistic at least as extreme as you observed.
- **Significance level** $\alpha$: a pre-chosen cutoff (commonly 0.05) used to decide whether to reject $H_0$.

### What a p-value is NOT
- It is **not** the probability $H_0$ is true.
- It is **not** the probability your model is correct.
- It is **not** a measure of economic importance.

### Type I / Type II errors and power
- **Type I error (false positive)**: rejecting $H_0$ when $H_0$ is true. Rough probability $\approx \alpha$ under assumptions.
- **Type II error (false negative)**: failing to reject $H_0$ when $H_1$ is true.
- **Power**: $1 - P(\text{Type II error})$. Power increases with larger samples and larger true effects.

### Regression t-test intuition
In OLS regression we estimate coefficients $\hat\beta$. A common test is:
- $H_0: \beta_j = 0$ (no linear association between $x_j$ and $y$ holding other features fixed)

The **t-statistic** is:

$$
t_j = \frac{\hat\beta_j - 0}{\widehat{SE}(\hat\beta_j)}
$$

Interpretation (roughly):
- if $|t_j|$ is large, $\hat\beta_j$ is far from 0 relative to its uncertainty estimate
- if $|t_j|$ is small, the data is compatible with $\beta_j$ being near 0 (given assumptions)

### Confidence intervals (CI) connect to hypothesis tests
A 95% CI is usually reported as:

$$
\hat\beta_j \pm t_{0.975}\cdot \widehat{SE}(\hat\beta_j)
$$

If the 95% CI does not include 0, the two-sided p-value is typically < 0.05.

### Python demo (toy): one-sample t-test and a regression coefficient test
This is not your project data; it is purely to make the objects concrete.

```python
import numpy as np
import pandas as pd
import statsmodels.api as sm
from scipy import stats

rng = np.random.default_rng(0)

# 1) One-sample t-test: is the mean of x equal to 0?
x = rng.normal(loc=0.2, scale=1.0, size=200)
t_stat, p_val = stats.ttest_1samp(x, popmean=0.0)
print("t-test t:", t_stat, "p:", p_val)

# 2) Regression t-test: is the slope on x equal to 0?
n = 300
x2 = rng.normal(size=n)
eps = rng.normal(scale=1.0, size=n)
y = 1.0 + 0.5 * x2 + eps

df = pd.DataFrame({"y": y, "x": x2})
X = sm.add_constant(df[["x"]])
res = sm.OLS(df["y"], X).fit()

# The summary includes coef, SE, t, p, and CI
print(res.summary())

# Manual t-stat for slope (matches summary output)
beta_hat = res.params["x"]
se_hat = res.bse["x"]
print("manual t:", beta_hat / se_hat)
```

### Common ways hypothesis testing goes wrong in ML + macro
- **Multiple testing**: you try many features/specifications; some will look “significant” by chance.
- **Violating assumptions**: autocorrelation and heteroskedasticity can make naive SE too small.
- **Confusing predictive success with causal claims**: a coefficient can predict well without being causal.

Practical guidance for this project:
- Report effect sizes + uncertainty, not just “significant / not significant.”
- For macro time series, prefer robust SE (HAC/Newey-West) when interpreting p-values.
- For predictive tasks, always complement p-values with out-of-sample evaluation.


<a id="load-census-data"></a>
## Load census data

### Goal
Load a county-level dataset created in the Census notebook.

If you haven't run the fetch notebook, use the bundled sample.



### Your Turn (1): Load processed county data (or sample)


In [ ]:
import pandas as pd

year = 2022  # TODO: set to the year you fetched
path = PROCESSED_DIR / f'census_county_{year}.csv'

if path.exists():
    df = pd.read_csv(path)
else:
    df = pd.read_csv(SAMPLE_DIR / 'census_county_sample.csv')

df.head()



### Your Turn (2): Inspect schema


In [ ]:
# TODO: Inspect columns and dtypes.
# Identify which columns represent:
# - income
# - rent
print(df.columns.tolist())
print(df.dtypes)
...



### Checkpoint (required columns)


In [ ]:
# TODO: Confirm the starter variables exist.
# If you used different ACS vars, update these names.
assert 'B19013_001E' in df.columns  # median household income
assert 'B25064_001E' in df.columns  # median gross rent
...



<a id="build-log-variables"></a>
## Build log variables

### Goal
Build a clean modeling table with:
- `log_income = log(income)`
- `log_rent = log(rent)`

Log-log regression is common in economics because it turns multiplicative relationships into additive ones.



### Your Turn (1): Clean and log-transform


In [ ]:
import numpy as np

# Raw variables
income = pd.to_numeric(df['B19013_001E'], errors='coerce')
rent = pd.to_numeric(df['B25064_001E'], errors='coerce')

# TODO: Drop non-positive values before taking logs
mask = (income > 0) & (rent > 0)
df_m = pd.DataFrame({
    'income': income[mask],
    'rent': rent[mask],
}).dropna()

# TODO: Create log variables
df_m['log_income'] = ...
df_m['log_rent'] = ...

df_m[['log_income', 'log_rent']].head()



### Your Turn (2): Visualize the relationship


In [ ]:
import matplotlib.pyplot as plt

# TODO: Make a scatter plot of log_income vs log_rent.
# Tip: use alpha=0.2 for dense plots.
...



<a id="fit-ols-hc3"></a>
## Fit OLS + HC3

### Goal
Fit an OLS regression and compute HC3 robust SE (common default for cross-sectional data).

Model:
$$\log(rent_i) = \beta_0 + \beta_1 \log(income_i) + \varepsilon_i$$



### Your Turn (1): Fit OLS + HC3


In [ ]:
from src import econometrics

# TODO: Fit OLS with HC3 robust SE using the helper.
res = econometrics.fit_ols_hc3(df_m, y_col='log_rent', x_cols=['log_income'])
print(res.summary())



### Your Turn (2): Extract the slope and interpret it


In [ ]:
# TODO: Extract coefficient and CI for log_income.
beta = float(res.params['log_income'])
ci = res.conf_int().loc['log_income'].tolist()

print('beta:', beta)
print('95% CI:', ci)

# Interpretation prompt:
# In a log-log model, beta is approximately an elasticity.
# Example: a 1% increase in income is associated with about beta% higher rent.
# TODO: Compute the implied change for a 10% income increase.
pct_income = 10.0
approx_pct_rent = ...
approx_pct_rent



<a id="interpretation"></a>
## Interpretation

Write a short interpretation (5-8 sentences) that answers:
- What is the estimated relationship?
- Is it statistically distinguishable from 0 (given assumptions)?
- Is it economically large?
- What would have to be true for a causal interpretation?



### Your Turn: Write your interpretation


In [ ]:
# TODO: Write your interpretation as a Python multiline string.
notes = """
...
"""
print(notes)



<a id="checkpoint-self-check"></a>
## Checkpoint (Self-Check)
Run a few asserts and write 2-3 sentences summarizing what you verified.



In [ ]:
# TODO: After you build X/y and split by time, validate the split.
# Example (adjust variable names):
# assert X_train.index.max() < X_test.index.min()
# assert y_train.index.equals(X_train.index)
# assert y_test.index.equals(X_test.index)
# assert not X_train.isna().any().any()
# assert not X_test.isna().any().any()
...



## Extensions (Optional)
- Try one additional variant beyond the main path (different features, different split, different model).
- Write down what improved, what got worse, and your hypothesis for why.



## Reflection
- What did you assume implicitly (about timing, availability, stationarity, or costs)?
- If you had to ship this model, what would you monitor?



<a id="solutions-reference"></a>
## Solutions (Reference)

Try the TODOs first. Use these only to unblock yourself or to compare approaches.

<details><summary>Solution: Load census data</summary>

_One possible approach. Your variable names may differ; align them with the notebook._

```python
# Reference solution for 00_single_factor_regression_micro — Load census data
import numpy as np
import pandas as pd

df = pd.read_csv(SAMPLE_DIR / 'census_county_sample.csv')
df.head()
```

</details>

<details><summary>Solution: Build log variables</summary>

_One possible approach. Your variable names may differ; align them with the notebook._

```python
# Reference solution for 00_single_factor_regression_micro — Build log variables
df = df.copy()
df = df[(df['B19013_001E'] > 0) & (df['B25064_001E'] > 0)].copy()
df['log_income'] = np.log(df['B19013_001E'].astype(float))
df['log_rent'] = np.log(df['B25064_001E'].astype(float))
```

</details>

<details><summary>Solution: Fit OLS + HC3</summary>

_One possible approach. Your variable names may differ; align them with the notebook._

```python
# Reference solution for 00_single_factor_regression_micro — Fit OLS + HC3
from src import econometrics

res = econometrics.fit_ols_hc3(df, y_col='log_rent', x_cols=['log_income'])
print(res.summary())
```

</details>

<details><summary>Solution: Interpretation</summary>

_One possible approach. Your variable names may differ; align them with the notebook._

```python
# Reference solution for 00_single_factor_regression_micro — Interpretation
# In a log-log model, the slope is an elasticity-style interpretation:
# a 1% increase in income is associated with ~beta% increase in rent (under assumptions).
```

</details>

